In [2]:
import os
import pandas as pd
import numpy as np
import cv2

def CheckDir(dirname):
	if not os.path.exists(dirname):
		os.makedirs(dirname)

DATA_DIR="Data/PP_pins"
IM_SIZE=(299,299)

In [ ]:
images=[]
img_path=[]
for dirname, _, filenames in os.walk(DATA_DIR):
	print(dirname, len(filenames))
	for filename in filenames:
		try:
			f_path=os.path.join(dirname, filename)
			images.append(cv2.imread(f_path))
			img_path.append(f_path)
		except Exception as e:
			print(filename, e)

In [17]:
pins=pd.DataFrame({
	"Path":img_path,
	"Images": images
})
pins.to_pickle("Data/pins.pkl")

In [2]:
pins=pd.read_pickle("Data/pins.pkl")
pins.head()

,Path,Images
0,Data/PP_pins/000337a0ee7432d5453c7016a757d410.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,Data/PP_pins/00201308aafb382364df742cd2c88367.jpg,"[[[84, 55, 94], [119, 80, 118], [115, 63, 100]..."
2,Data/PP_pins/00264f7f0e84a86fddd2dc390941c044.jpg,"[[[216, 237, 228], [214, 238, 228], [209, 233,..."
3,Data/PP_pins/0027d4a15fbca7694c80a3a96383292c.jpg,"[[[77, 63, 14], [78, 64, 15], [78, 64, 15], [7..."
4,Data/PP_pins/0028f6bcf10b0b32b8e5774fc21cc305.jpg,"[[[182, 188, 187], [184, 191, 188], [186, 189,..."


In [12]:
from keras.applications.inception_v3 import InceptionV3

MODEL_DIR="../dgx/models/"

model=InceptionV3(weights=MODEL_DIR+"inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5",
				 include_top=False,
				 pooling='avg')

In [17]:
features=model.predict(np.stack(pins['Images'].values))

In [25]:
FEATURE_DIR="Data/features/"
CheckDir(FEATURE_DIR)
np.save(FEATURE_DIR+"Inceptionv3_avg.npy",features)

In [4]:
FEATURE_DIR="Data/features/"
features=np.load(FEATURE_DIR+"Inceptionv3_avg.npy")